In [ ]:
!pip install cartopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 36.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import  xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import warnings
import math
from tqdm.auto import tqdm

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cartopy.feature as cf
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker

from cartopy.util import add_cyclic_point
from matplotlib import animation
from tqdm.auto import tqdm
from datetime import datetime,timedelta

from matplotlib.cm import get_cmap
warnings.filterwarnings("ignore")

The **climatology** for **pre-monsoon (MAM)** and **monsoon (JJAS)** season was derived from ERA5 **500hpa Vertical velocity**  data (1981–2010)

In [ ]:
ds=xr.open_mfdataset('/content/drive/MyDrive/AP_HW/ERA5 Data/vertical_velocity_500/vv_500-1971.nc')
ds

<xarray.Dataset> Size: 930MB
Dimensions:         (valid_time: 2920, pressure_level: 1, latitude: 209,
                     longitude: 381)
Coordinates:
    number          int64 8B ...
  * valid_time      (valid_time) datetime64[ns] 23kB 1971-01-01 ... 1971-12-3...
  * pressure_level  (pressure_level) float64 8B 500.0
  * latitude        (latitude) float64 2kB 52.0 51.75 51.5 ... 0.5 0.25 0.0
  * longitude       (longitude) float64 3kB 25.0 25.25 25.5 ... 119.8 120.0
    expver          (valid_time) <U4 47kB dask.array<chunksize=(2920,), meta=np.ndarray>
Data variables:
    w               (valid_time, pressure_level, latitude, longitude) float32 930MB dask.array<chunksize=(974, 1, 70, 127), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-12-25T16:33 GRIB to CDM+CF via cfgrib-0.9.1...

In [ ]:
#@title Climatology Calculation - MAM
import os
from tqdm.auto import tqdm
import datetime

# Print process start time
print(datetime.datetime.utcnow() + datetime.timedelta(hours=6))

# List to store yearly datasets
vv_list = []

directory_path = '/content/drive/MyDrive/AP_HW/ERA5 Data/vertical_velocity_500'

# Loop over files
for filename in tqdm(sorted(os.listdir(directory_path))):
    if filename.endswith('.nc'):
        # Open dataset
        file_path = os.path.join(directory_path, filename)
        year = filename[-7:-3]
        vv = xr.open_dataset(file_path)

        # Define MAM period
        start_time = str(np.datetime64(f'{year}-03-01T00:00:00.000000000') - np.timedelta64(6, 'h'))
        end_time = str(np.datetime64(f'{year}-05-31T17:00:00.000000000'))
        time_range = slice(start_time, end_time)

        # Select time and pressure level
        vv = vv.sel(valid_time=time_range, pressure_level=500)

        # Adjust time to Bangladesh timezone
        vv['valid_time'] = vv['valid_time'] + np.timedelta64(6, 'h')

        # Compute yearly mean
        ds_vv = vv.groupby('valid_time.year').mean(dim='valid_time', skipna=True)

        # Add to list
        vv_list.append(ds_vv)

# Concatenate all years
vv_ds = xr.concat(vv_list, dim='year')

# Compute climatology (1981-2010)
vv_clim_ds = vv_ds.sel(year=slice(1981, 2010))
vv_climatology = vv_clim_ds.mean(dim='year', skipna=True)

# Save to NetCDF
vv_climatology.to_netcdf('/content/drive/MyDrive/AP_HW/Scripts-ll/climatology/data/vv_500_mam_climatology.nc')

# Print process end time
print(datetime.datetime.utcnow() + datetime.timedelta(hours=6))


2025-06-21 10:22:15.480569


  0%|          | 0/54 [00:00<?, ?it/s]

2025-06-21 10:34:31.441747


In [ ]:
#@title Climatology Calculation - JJAS
import os
from tqdm.auto import tqdm
import datetime

# Print process start time
print(datetime.datetime.utcnow() + datetime.timedelta(hours=6))

# List to store yearly datasets
vv_list = []

directory_path = '/content/drive/MyDrive/AP_HW/ERA5 Data/vertical_velocity_500'

# Loop over files
for filename in tqdm(sorted(os.listdir(directory_path))):
    if filename.endswith('.nc'):
        # Open dataset
        file_path = os.path.join(directory_path, filename)
        year = filename[-7:-3]
        vv = xr.open_dataset(file_path)

        # Define JJAS period
        start_time = str(np.datetime64(f'{year}-06-01T00:00:00.000000000') - np.timedelta64(6, 'h'))
        end_time = str(np.datetime64(f'{year}-09-30T17:00:00.000000000'))
        time_range = slice(start_time, end_time)

        # Select time and pressure level
        vv = vv.sel(valid_time=time_range, pressure_level=500)

        # Adjust time to Bangladesh timezone
        vv['valid_time'] = vv['valid_time'] + np.timedelta64(6, 'h')

        # Compute yearly mean
        ds_vv = vv.groupby('valid_time.year').mean(dim='valid_time', skipna=True)

        # Add to list
        vv_list.append(ds_vv)

# Concatenate all years
vv_ds = xr.concat(vv_list, dim='year')

# Compute climatology (1981-2010)
vv_clim_ds = vv_ds.sel(year=slice(1981, 2010))
vv_climatology = vv_clim_ds.mean(dim='year', skipna=True)

# Save to NetCDF
vv_climatology.to_netcdf('/content/drive/MyDrive/AP_HW/Scripts-ll/climatology/data/vv_500_jjas_climatology.nc')

# Print process end time
print(datetime.datetime.utcnow() + datetime.timedelta(hours=6))


2025-06-21 10:34:31.468080


  0%|          | 0/54 [00:00<?, ?it/s]

2025-06-21 10:46:55.604219
